In [1]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler

import sys
sys.path.append("../src")
import FE
import dataset
import models
import util
from config import read_config, updateConfig
import train_helper
import infer_helper
import loss

2021-10-13 07:55:08.114338: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-13 07:55:08.114367: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [4]:
BaseConfig = read_config("base")
BaseConfig.debug=True
BaseConfig.batch_size = 512
BaseConfig = updateConfig(BaseConfig)

Using device: cuda
Number of device: 1
Model Output Folder: ../output/BasicTest/


In [5]:
train, test = dataset.read_data(BaseConfig)
display(train.shape, test.shape)
display(train.head())
display(test.head())

(102400, 9)

(102400, 7)

,id,breath_id,R,C,time_step,u_in,u_out,pressure,fold
0,1,1,20,50,0.000000,0.083334,0,5.837492,0
1,2,1,20,50,0.033652,18.383041,0,5.907794,4
2,3,1,20,50,0.067514,22.509277,0,7.876254,2
3,4,1,20,50,0.101542,22.808823,0,11.742872,0
4,5,1,20,50,0.135756,25.355850,0,12.234987,2


,id,breath_id,R,C,time_step,u_in,u_out
0,1,0,5,20,0.000000,0.000000,0
1,2,0,5,20,0.031904,7.515046,0
2,3,0,5,20,0.063827,14.651675,0
3,4,0,5,20,0.095751,21.230610,0
4,5,0,5,20,0.127644,26.320955,0


In [6]:
train["u_out"]=0

In [7]:
# y_train_fake = np.array(list(range(30))).reshape(-1, 5)
# w_train_fake = np.array([1]*5 + [0]*25).reshape(-1, 5)
# y_pred_fake = np.array([0,1,2,3,5] + [0] * 25).reshape(-1,5)
# loss.loss_fnc(torch.tensor(y_train_fake), torch.tensor(y_pred_fake), torch.tensor(w_train_fake))
# util.get_score(y_train_fake, y_pred_fake, w_train_fake)

In [8]:
train_helper.training_loop(train.copy(), BaseConfig)

Fold:  0
Prepare train valid
(82000, 9) (20400, 9)
(82000, 22)
training data samples, val data samples:  1025 255
Model Size: 5649765
Epoch:  0


RuntimeError: cuDNN error: CUDNN_STATUS_EXECUTION_FAILED

In [7]:
test_avg = infer_helper.get_test_avg(train, test, BaseConfig)

Prepare train valid
(102400, 9) (102400, 8)
(102400, 51)


100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:06<00:00,  6.58s/it]


count    102400.000000
mean          0.954662
std           0.048827
min           0.599037
25%           0.955804
50%           0.973450
75%           0.984142
max           0.988415
Name: pressure, dtype: float64
test file saved to: ../output/BasicTest//submission.csv
